In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.8.0'

In [3]:
df = pd.read_csv("Churn_Modelling.csv")

In [4]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
X = df.iloc[:,3:-1].values
y = df.iloc[:,-1].values

In [8]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [9]:
y

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

In [13]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] =le.fit_transform(X[:,2])

In [21]:
X

array([[1.0, 0.0, 1.0, ..., 1, 1, 101348.88],
       [1.0, 0.0, 0.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 1.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 1.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 1.0, ..., 1, 0, 38190.78]], dtype=object)

In [26]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder',OneHotEncoder(), [1])],remainder = 'passthrough')
X = ct.fit_transform(X)

In [27]:
X

array([[1.0, 0.0, 1.0, ..., 1, 1, 101348.88],
       [1.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 1.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 1.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 1.0, ..., 1, 0, 38190.78]], dtype=object)

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [30]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [33]:
ann = tf.keras.models.Sequential()

ann.add(tf.keras.layers.Dense(6, activation= 'relu'))
ann.add(tf.keras.layers.Dense(6,activation='relu'))

ann.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

ann.compile(loss = 'binary_crossentropy',
           optimizer = 'adam',
           metrics = ['accuracy'])

ann.fit(X_train,y_train, batch_size=32, epochs=50)

Epoch 1/50
250/250 [==============================] - 1s 776us/step - loss: 0.5650 - accuracy: 0.7380
Epoch 2/50
250/250 [==============================] - 0s 789us/step - loss: 0.4618 - accuracy: 0.7971
Epoch 3/50
250/250 [==============================] - 0s 840us/step - loss: 0.4355 - accuracy: 0.8075
Epoch 4/50
250/250 [==============================] - 0s 844us/step - loss: 0.4181 - accuracy: 0.8202
Epoch 5/50
250/250 [==============================] - 0s 815us/step - loss: 0.4037 - accuracy: 0.8309
Epoch 6/50
250/250 [==============================] - 0s 763us/step - loss: 0.3900 - accuracy: 0.8384
Epoch 7/50
250/250 [==============================] - 0s 759us/step - loss: 0.3772 - accuracy: 0.8472
Epoch 8/50
250/250 [==============================] - 0s 763us/step - loss: 0.3675 - accuracy: 0.8518
Epoch 9/50
250/250 [==============================] - 0s 767us/step - loss: 0.3613 - accuracy: 0.8528
Epoch 10/50
250/250 [==============================] - 0s 772us/step - loss: 0.356

In [39]:
y_pred = ann.predict(X_test)
y_pred = y_pred > 0.5
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [62]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm=confusion_matrix(y_test,y_pred)
print(cm)
print(f'accuracy score : {accuracy_score(y_test,y_pred)}')

[[1519   76]
 [ 197  208]]
accuracy score : 0.8635
